# Parte 3: Recolección de información
### Web Scraping a "Semana" para investigación referente a los cambios climáticos en Colombia

### Paso 1: Importación de librerias

In [1]:
import pandas as pd
import time
import random
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle

### Paso 2: Definición de opciones para el navegador

In [2]:
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

### Paso 3: Definición de clase personalizada

In [3]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejeción"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

### Paso 4: Lectura de archivo de noticias

In [4]:
# Definición de la clase
class IterableClass:
    
    # Función inicial o constructor
    def __init__(self, list_values = None): # None es para que sea opcional
        self.list_values = list_values
        self.dateframe = pd.read_csv(r'df_clean_4.csv') # lectura del archivo
        self.i = 0
        
    # Retornar un iterador
    def __iter__(self):
        return self
    
    # Retorna un elemento del contenedor
    def __next__(self):
        if self.i >= len(self.list_values):
            print("\nNo hay más valores a recorrer... ⬇")
            raise StopIteration
            
        item = self.list_values[self.i]
        self.i += 1
        return item  

### Paso 5: Instacia del objeto e impresión del DataFrame

In [5]:
# Instancia del objeto
IC = IterableClass()

# 5.1. Eliminación de columna que se creo en por error "Unnamed: 0" .drop('Unnamed: 0', axis=1)
# 5.2. Reseteo de index por la acción de combinación de DataFrames .reset_index().drop(['index'], axis=1)
IC.dateframe.drop('Unnamed: 0', axis=1).reset_index().drop(['index'], axis=1)

,headers_text,headers_link
0,"No todos critican a Trump\r\nvie, 2 jun 2017\r...",https://www.semana.com/putin-y-bjorn-lomborg-n...
1,¿Por qué Nicaragua no firma el Acuerdo de Parí...,https://www.semana.com/nicaragua-no-firma-el-a...
2,“El futuro del humano depende de los primates”...,https://www.semana.com/sam-shanee-de-npc-el-fu...
3,El increíble descubrimiento de cangrejos caver...,https://www.semana.com/cangrejos-cavernicolas-...
4,"La Tierra pide ayuda en su día\r\nvie, 21 abr ...",https://www.semana.com/dia-de-la-tierra-2017-e...
...,...,...
295,Cuatro playas colombianas fueron certificadas ...,https://www.semana.com/medio-ambiente/articulo...
296,"Hidroituango, deforestación y líderes amenazad...",https://www.semana.com/hidroituango-deforestac...
297,Así combatirá la deforestación el gobierno de ...,https://www.semana.com/medio-ambiente/articulo...
298,Emergencia ambiental por mortandad de peces en...,https://www.semana.com/emergencia-ambiental-po...


### Paso 6: Definición de función para unificar los diferentes parrafos de la noticia

In [6]:
def JoinParagraphs(paragraphs):
    TextParagraphs = ""
    for paragraph in paragraphs:
        TextParagraphs = TextParagraphs + paragraph.text
        
    return TextParagraphs

### Paso 7: Definición de función para unificar los diferentes temas de la noticia

In [7]:
def JoinThemes(themes):
    TextThemes = ""
    for theme in themes:
        TextThemes = TextThemes + theme.text + ", "
    return TextThemes

### Paso 8: Recolección de información para cada noticia

In [8]:
# 8.1 Declaraciones de variables
titles = []
summaries = []
authors = []
dates = []
paragraphs = []
themes = []

In [9]:
# PASO 8.2: Aviso para el usuario final de que se inicio el web-scraping --------------------------------------------------------------#
print("Se inició el Web Scraping a 'Semana', por favor espere...") # Mensaje para visualización del inicio del proceso

# 8.2.1 Variable para definir el inicio de tiempo en ejecución
start = time.time()

Se inició el Web Scraping a 'Semana', por favor espere...


In [10]:
# PASO 8.3 Estructura base para acceder a la página en cuestión ----------------------------------------------------------------------#
service = Service(executable_path=r'../../chromedriver-win64/chromedriver.exe') # Creación del servicio mediante el driver
options = webdriver.ChromeOptions() # Definición de variable con las opciones para chrome
driver = webdriver.Chrome(service=service, options=options) # Creación del objeto del WebDriver

# Asignación de dataframe a una variable
links = IC.dateframe

# Control del error al momento de finalizar el páginado
try:
    
    # Iteración para acceso a los links del dataframe
    for i in range(len(links)):
        
        news = {
            "titles" : [],
            "summaries" : [],
            "authors" : [],
            "dates" : [],
            "paragraphs" : [],
            "themes" : [],
        }
        
        # Acceso a cada link del dataframe
        driver.get(links.iloc[i]['headers_link'])

        # Títulos
        # ****************************************************************************************************************
        title = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/h1')
        if title:
            for i in title:
                print(titles.append(i.text))
        else: titles.append('...')
        # ****************************************************************************************************************

        # Resúmenes
        # ****************************************************************************************************************
        summary = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/p')
        if summary: 
            for i in summary:
                print(summaries.append(i.text))
        else: summaries.append('...')
        # ****************************************************************************************************************

        # Autor
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                author_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/a/div')
                if author_a:
                    for i in author_a:
                        print(authors.append(i.text))
                else: authors.append('...')
            elif len(date_a) > 1:
                author_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[1]')
                if author_b:
                    for i in author_b:
                        print(authors.append(i.text))
                else: authors.append('...')
            else: authors.append('...')
        else: authors.append('...')
        # Test
        author_a = None
        author_b = None
        date_a = None
        # ****************************************************************************************************************

        # Fecha
        # ****************************************************************************************************************
        date_a = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div')
        if date_a: 
            if len(date_a) == 1:
                if date_a:
                    for i in date_a:
                        print(dates.append(i.text))
                else: dates.append('...')
            elif len(date_a) > 1:
                date_b = driver.find_elements(By.XPATH, '//*[@id="fusion-app"]/div/main/article/div/div[2]/div[2]/div[2]')
                if date_b:
                    for i in date_b:
                        print(dates.append(i.text))
                else: dates.append('...')
            else: dates.append('...')
        else: dates.append('...')
        # Test
        author_a = None
        date_a = None
        date_b = None
        # ****************************************************************************************************************
        
        # Parrafos
        # ****************************************************************************************************************
        paragraphs_a = driver.find_elements(By.CLASS_NAME, 'prose')
        if paragraphs_a: 
            if len(paragraphs_a) > 1:
                paragraphs.append(JoinParagraphs(paragraphs_a))
            else:
                paragraphs.append(paragraphs_a[0].text)
        else: paragraphs.append('...')
        # ****************************************************************************************************************

        # Temas
        # ****************************************************************************************************************
        tags = driver.find_elements(By.CSS_SELECTOR, 'a.inline-block')
        if tags: 
            if len(tags) > 1:
                themes.append(JoinThemes(tags))
            else:
                themes.append(tags[0].text)
        else: themes.append('...')
        # ****************************************************************************************************************
                
        # Construcción del diccionario 
        news = {
            "titles" : titles,
            "summaries" : summaries,
            "authors" : authors,
            "dates" : dates,
            "paragraphs" : paragraphs,
            "themes" : themes
        }

        # Construcción inicial del dataframe
        df = pd.DataFrame(news)

        # (Opcional) Modo DEBUG, sirve para validar como el DataFrame se está llenando según las iteraciones
        display(df)
    else:
        driver.quit();
        
except Exception as e:
    display("Exception :", e)
    driver.quit() 
except NoSuchElementException as nse:
    display("NoSuchElementException :", nse)
    driver.quit()
except TimeoutException as tim:
    display("TimeoutException :", tim)
    driver.quit()

None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,No todos critican a Trump,Vladimir Putin y Bjorn Lomborg suavizan la llu...,...,2 de junio de 2017,El anuncio de la salida de Estados Unidos del ...,"Bjorn Lomborg, Acuerdo de París, Donald Trump,..."


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,No todos critican a Trump,Vladimir Putin y Bjorn Lomborg suavizan la llu...,...,2 de junio de 2017,El anuncio de la salida de Estados Unidos del ...,"Bjorn Lomborg, Acuerdo de París, Donald Trump,..."
1,¿Por qué Nicaragua no firma el Acuerdo de París?,Nicaragua es uno de los pocos países que no ha...,...,8 de junio de 2017,Cuando las delegaciones de negociación de todo...,"Nicaragua, Acuerdo de París, Cambio climático,..."


None
None
None


...

,titles,summaries,authors,dates,paragraphs,themes
0,No todos critican a Trump,Vladimir Putin y Bjorn Lomborg suavizan la llu...,...,2 de junio de 2017,El anuncio de la salida de Estados Unidos del ...,"Bjorn Lomborg, Acuerdo de París, Donald Trump,..."
1,¿Por qué Nicaragua no firma el Acuerdo de París?,Nicaragua es uno de los pocos países que no ha...,...,8 de junio de 2017,Cuando las delegaciones de negociación de todo...,"Nicaragua, Acuerdo de París, Cambio climático,..."
2,“El futuro del humano depende de los primates”,Aunque muchos hablan de la posible extinción d...,...,8 de junio de 2017,Hace unas semanas la prestigiosa revista Journ...,"Primates, Monos, Neotropical Primate Conservat..."
3,El increíble descubrimiento de cangrejos caver...,El Instituto Humboldt confirma el hallazgo de ...,...,15 de junio de 2017,Se trata de tres hembras y un macho Neostrenge...,"Cangrejos cavernícolas, Instituto Humboldt, Ci..."
4,La Tierra pide ayuda en su día,El Día Internacional de la Tierra es cada año ...,...,21 de abril de 2017,"Más que una bonita conmemoración, el 22 de abr...","Día de la Tierra, Planeta Tierra, ONU, Conserv..."
...,...,...,...,...,...,...
293,EN VIDEO: Incendio arrasa con el páramo de Cum...,Desde hace dos días se registra una conflagrac...,...,11 de noviembre de 2020,Páramo de Cumbal en Nariño\nNoticias de Colomb...,"Páramo de Cumbal en Nariño, Noticias de Colomb..."
294,Denuncian que petrolera entregó informes errad...,Autoridades de Puerto Boyacá indicaron que las...,...,5 de octubre de 2020,Derrame de petróleo en Puerto Boyacá\nPuerto B...,"Derrame de petróleo en Puerto Boyacá, Puerto B..."
295,Cuatro playas colombianas fueron certificadas ...,"Las playas del Hotel Magüipi en Buenaventura, ...",...,3 de octubre de 2019,Unirme al canal de WhatsApp.com de Semana Noti...,...
296,"Hidroituango, deforestación y líderes amenazad...",El proyecto hidroeléctrico más grande de Colom...,Antonio José Paz Cardona/Mongabay Latam,19 de diciembre de 2018,Deforestación\nIndígenas\nHidroituango\nPáramo...,"Deforestación, Indígenas, Hidroituango, Páramo..."


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,No todos critican a Trump,Vladimir Putin y Bjorn Lomborg suavizan la llu...,...,2 de junio de 2017,El anuncio de la salida de Estados Unidos del ...,"Bjorn Lomborg, Acuerdo de París, Donald Trump,..."
1,¿Por qué Nicaragua no firma el Acuerdo de París?,Nicaragua es uno de los pocos países que no ha...,...,8 de junio de 2017,Cuando las delegaciones de negociación de todo...,"Nicaragua, Acuerdo de París, Cambio climático,..."
2,“El futuro del humano depende de los primates”,Aunque muchos hablan de la posible extinción d...,...,8 de junio de 2017,Hace unas semanas la prestigiosa revista Journ...,"Primates, Monos, Neotropical Primate Conservat..."
3,El increíble descubrimiento de cangrejos caver...,El Instituto Humboldt confirma el hallazgo de ...,...,15 de junio de 2017,Se trata de tres hembras y un macho Neostrenge...,"Cangrejos cavernícolas, Instituto Humboldt, Ci..."
4,La Tierra pide ayuda en su día,El Día Internacional de la Tierra es cada año ...,...,21 de abril de 2017,"Más que una bonita conmemoración, el 22 de abr...","Día de la Tierra, Planeta Tierra, ONU, Conserv..."
...,...,...,...,...,...,...
294,Denuncian que petrolera entregó informes errad...,Autoridades de Puerto Boyacá indicaron que las...,...,5 de octubre de 2020,Derrame de petróleo en Puerto Boyacá\nPuerto B...,"Derrame de petróleo en Puerto Boyacá, Puerto B..."
295,Cuatro playas colombianas fueron certificadas ...,"Las playas del Hotel Magüipi en Buenaventura, ...",...,3 de octubre de 2019,Unirme al canal de WhatsApp.com de Semana Noti...,...
296,"Hidroituango, deforestación y líderes amenazad...",El proyecto hidroeléctrico más grande de Colom...,Antonio José Paz Cardona/Mongabay Latam,19 de diciembre de 2018,Deforestación\nIndígenas\nHidroituango\nPáramo...,"Deforestación, Indígenas, Hidroituango, Páramo..."
297,Así combatirá la deforestación el gobierno de ...,Operativos para luchar contra las mafias defor...,...,23 de octubre de 2018,Presidente de Colombia\nIván Duque\nMinisterio...,"Presidente de Colombia, Iván Duque, Ministerio..."


None
None
None


,titles,summaries,authors,dates,paragraphs,themes
0,No todos critican a Trump,Vladimir Putin y Bjorn Lomborg suavizan la llu...,...,2 de junio de 2017,El anuncio de la salida de Estados Unidos del ...,"Bjorn Lomborg, Acuerdo de París, Donald Trump,..."
1,¿Por qué Nicaragua no firma el Acuerdo de París?,Nicaragua es uno de los pocos países que no ha...,...,8 de junio de 2017,Cuando las delegaciones de negociación de todo...,"Nicaragua, Acuerdo de París, Cambio climático,..."
2,“El futuro del humano depende de los primates”,Aunque muchos hablan de la posible extinción d...,...,8 de junio de 2017,Hace unas semanas la prestigiosa revista Journ...,"Primates, Monos, Neotropical Primate Conservat..."
3,El increíble descubrimiento de cangrejos caver...,El Instituto Humboldt confirma el hallazgo de ...,...,15 de junio de 2017,Se trata de tres hembras y un macho Neostrenge...,"Cangrejos cavernícolas, Instituto Humboldt, Ci..."
4,La Tierra pide ayuda en su día,El Día Internacional de la Tierra es cada año ...,...,21 de abril de 2017,"Más que una bonita conmemoración, el 22 de abr...","Día de la Tierra, Planeta Tierra, ONU, Conserv..."
...,...,...,...,...,...,...
295,Cuatro playas colombianas fueron certificadas ...,"Las playas del Hotel Magüipi en Buenaventura, ...",...,3 de octubre de 2019,Unirme al canal de WhatsApp.com de Semana Noti...,...
296,"Hidroituango, deforestación y líderes amenazad...",El proyecto hidroeléctrico más grande de Colom...,Antonio José Paz Cardona/Mongabay Latam,19 de diciembre de 2018,Deforestación\nIndígenas\nHidroituango\nPáramo...,"Deforestación, Indígenas, Hidroituango, Páramo..."
297,Así combatirá la deforestación el gobierno de ...,Operativos para luchar contra las mafias defor...,...,23 de octubre de 2018,Presidente de Colombia\nIván Duque\nMinisterio...,"Presidente de Colombia, Iván Duque, Ministerio..."
298,Emergencia ambiental por mortandad de peces en...,Corporaciones autónomas emitieron una alerta p...,...,24 de marzo de 2019,Mortandad de peces\nRío Lengupá\nBoyacá\nCasan...,"Mortandad de peces, Río Lengupá, Boyacá, Casan..."


In [11]:
df_news_clean = df.drop_duplicates(subset=["titles"])

In [12]:
df_news_clean.to_csv("df_news/df_news_clean_4.csv")

In [13]:
# Variable para definir el final de tiempo en ejecución
end = time.time()

# Variable para calcular tiempo y para enviar a la función
result = end-start

#Impresión del tiempo que tardó
print(Seconds(result))

El sitema tardó: 0 horas 24 minutos 7 segundos en finalizar su ejeción


In [14]:
df_news_clean

,titles,summaries,authors,dates,paragraphs,themes
0,No todos critican a Trump,Vladimir Putin y Bjorn Lomborg suavizan la llu...,...,2 de junio de 2017,El anuncio de la salida de Estados Unidos del ...,"Bjorn Lomborg, Acuerdo de París, Donald Trump,..."
1,¿Por qué Nicaragua no firma el Acuerdo de París?,Nicaragua es uno de los pocos países que no ha...,...,8 de junio de 2017,Cuando las delegaciones de negociación de todo...,"Nicaragua, Acuerdo de París, Cambio climático,..."
2,“El futuro del humano depende de los primates”,Aunque muchos hablan de la posible extinción d...,...,8 de junio de 2017,Hace unas semanas la prestigiosa revista Journ...,"Primates, Monos, Neotropical Primate Conservat..."
3,El increíble descubrimiento de cangrejos caver...,El Instituto Humboldt confirma el hallazgo de ...,...,15 de junio de 2017,Se trata de tres hembras y un macho Neostrenge...,"Cangrejos cavernícolas, Instituto Humboldt, Ci..."
4,La Tierra pide ayuda en su día,El Día Internacional de la Tierra es cada año ...,...,21 de abril de 2017,"Más que una bonita conmemoración, el 22 de abr...","Día de la Tierra, Planeta Tierra, ONU, Conserv..."
...,...,...,...,...,...,...
295,Cuatro playas colombianas fueron certificadas ...,"Las playas del Hotel Magüipi en Buenaventura, ...",...,3 de octubre de 2019,Unirme al canal de WhatsApp.com de Semana Noti...,...
296,"Hidroituango, deforestación y líderes amenazad...",El proyecto hidroeléctrico más grande de Colom...,Antonio José Paz Cardona/Mongabay Latam,19 de diciembre de 2018,Deforestación\nIndígenas\nHidroituango\nPáramo...,"Deforestación, Indígenas, Hidroituango, Páramo..."
297,Así combatirá la deforestación el gobierno de ...,Operativos para luchar contra las mafias defor...,...,23 de octubre de 2018,Presidente de Colombia\nIván Duque\nMinisterio...,"Presidente de Colombia, Iván Duque, Ministerio..."
298,Emergencia ambiental por mortandad de peces en...,Corporaciones autónomas emitieron una alerta p...,...,24 de marzo de 2019,Mortandad de peces\nRío Lengupá\nBoyacá\nCasan...,"Mortandad de peces, Río Lengupá, Boyacá, Casan..."
